In [1]:
!pip install gym==0.21 > /dev/null
!pip install stable-baselines3[extra] box2d > /dev/null

In [2]:
!pip install pyvirtualdisplay > /dev/null
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null

In [3]:
import random 
import gym
import shutil
from stable_baselines3 import DQN
from collections import OrderedDict

In [4]:
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline
import glob
import io
import base64
from gym.wrappers import Monitor
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [5]:
env = gym.make("CartPole-v1")

In [6]:
model = DQN(policy='MlpPolicy', env=env, batch_size=64, buffer_size=100000, 
            exploration_final_eps=0.04, exploration_fraction=0.16, gamma=0.99, 
            gradient_steps=128, learning_rate=0.0023, learning_starts=1000, 
            policy_kwargs=dict(net_arch=[256, 256]), target_update_interval=10, 
            train_freq=256)
model.learn(total_timesteps=50000)
model.save("cart_pole_dqn")
# model.load("cart_pole_dqn")

In [7]:
def show_video(folder):
  mp4list = glob.glob(f'{folder}/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env, folder):
  env = Monitor(env, folder, force=True)
  return env

In [8]:
def simulate(env, model, verbose=False):
  observation = env.reset()
  done = False
  i = 0
  while not done:
    i += 1
    action = model.predict(observation)[0]
    observation, reward, done, info = env.step(action)
    if verbose and (done or i % 500):
      display_results(observation, reward, done, info)
  return i, reward
      

def display_results(observation, reward, done, info):
  print(f"Observation: {observation}")
  print(f"Reward: {reward}")
  print(f"Done: {done}")
  print(f"Info: {info}")
  print("==================================================")



---



In [9]:
# Show env.step output
observation = env.reset()
action = model.predict(observation)[0]
observation, reward, done, info = env.step(action)
display_results(observation, reward, done, info)
env.close()

Observation: [ 0.03379505  0.23307054 -0.00368966 -0.2831098 ]
Reward: 1.0
Done: False
Info: {}




---



In [ ]:
video_folder = './video'
env = wrap_env(env, video_folder)
steps, rewards = simulate(env, model)
env.close()
show_video(video_folder)

In [11]:
best_reward = -1e8
for i in tqdm(range(100)):
  env = wrap_env(env, './video')
  steps, reward = simulate(env, model)
  env.close()
  if reward > best_reward:
    best_reward = reward
    shutil.copytree('./video', f'./video_{best_reward}') 
print(best_reward)


100%|██████████| 100/100 [07:28<00:00,  4.49s/it]

1.0




---



In [12]:
print(best_reward)
show_video(f'video_{best_reward}')

# number = 62
# show_video(f'video_{number}')

1.0
